In [1]:
import os
import conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(conda_dir, 'Library\share')
# proj_lib = os.path.join(os.path.join(conda_dir, 'pkgs'), 'proj4-5.2.0-h6538335_1006\Library\share')
path_gdal = os.path.join(proj_lib, 'gdal')
os.environ ['PROJ_LIB']=proj_lib
os.environ ['GDAL_DATA']=path_gdal

from datetime import datetime
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon

In [3]:
gdf_bui = gpd.read_file('buildings_1_СТН.json',encoding='utf-8')

In [7]:
gdf_addr = gpd.read_file('gdf_pt.json',encoding='utf-8')

In [9]:
adresses = gdf_addr[['address','total_sq','total_ppl','location','geometry']]

In [16]:
adresses = adresses.rename(columns={'address':'addr_ref'})

In [43]:
bui_buff = gdf_bui.copy()
bui_buff.crs='epsg:4326'
bui_buff.geometry = bui_buff.geometry.to_crs('epsg:32637').buffer(30).to_crs('epsg:4326')
# bui_buff.geometry = bui_buff.geometry.buffer(30)
# bui_buff = bui_buff.to_crs('epsg:4326')

In [47]:
bui_buff.to_file('bui_buff.json',driver='GeoJSON',encoding='utf-8')

In [71]:
bui_buff.columns

Index(['osm_id', 'address', 'bui_lvl', 'name', 'amenity', 'bui_type', 'tags',
       'total_area_m', 'bui_group', 'city_name', 'geometry'],
      dtype='object')

In [72]:
time_end = "{:%H:%M:%S}".format(datetime.now())
print("time start:", time_end)
time_all = round(len(adresses) / 17,2)
print("Estimated time:", time_all, "seconds")
sj_poly = gpd.sjoin(bui_buff[['osm_id','geometry']], adresses).drop('index_right',axis=1)
#
time_end = "{:%H:%M:%S}".format(datetime.now())
print("time end:", time_end)

time start: 16:56:57
Estimated time: 20.82 seconds
time end: 16:57:16


In [73]:
sj_poly.columns

Index(['osm_id', 'geometry', 'addr_ref', 'total_sq', 'total_ppl', 'location'], dtype='object')

In [74]:
# bui_new = gdf_bui[gdf_bui.osm_id.isin(sj_poly.osm_id)]
bui_new = gdf_bui.merge(sj_poly[['osm_id', 'addr_ref', 'total_sq',
       'total_ppl', 'location']],how='left',on=['osm_id'])

# bui_new['geometry'] = bui_new['geometry_x']
# del bui_new['geometry_y'], bui_new['geometry_x']
bui_new = gpd.GeoDataFrame(bui_new)
bui_new.crs='epsg:4326'

In [75]:
print(len(gdf_bui))
print(len(bui_new))

191563
191767


In [60]:
not_found = adresses[~adresses.addr_ref.isin(bui_new.addr_ref)].reset_index(drop=True)
print(len(not_found))

8


In [37]:
not_found.to_file('not_found.json',driver='GeoJSON',encoding='utf-8')

In [38]:
sj_poly.to_file('sj_poly.json',driver='GeoJSON',encoding='utf-8')